In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations,combinations
from collections import Counter

In [2]:
#creating a function that finds pairs and list them in a column name as A and B
def find_pairs(x):
    pairs = pd.DataFrame(list(permutations(x.values,2)),columns=["A","B"])
    return pairs

In [3]:
#Load in the dataset and check the head of the data
dataset=pd.read_csv('sales all data.csv')
dataset.head()

,Source.Name,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,Sales_April_2019.csv,176558.0,USB-C Charging Cable,2.0,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,Sales_April_2019.csv,NaN,NaN,NaN,NaN,NaN,NaN
2,Sales_April_2019.csv,176559.0,Bose SoundSport Headphones,1.0,99.99,04-07-2019 22:30,"682 Chestnut St, Boston, MA 02215"
3,Sales_April_2019.csv,176560.0,Google Phone,1.0,600.00,04-12-2019 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,Sales_April_2019.csv,176560.0,Wired Headphones,1.0,11.99,04-12-2019 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [5]:
#Group the products by Order ID then apply the function 
dataset_combo =dataset.groupby('Order ID')['Product'].apply(find_pairs).reset_index(drop=True)

In [15]:
dataset_combo.head()

,A,B
0,USB-C Charging Cable,Wired Headphones
1,Wired Headphones,USB-C Charging Cable
2,Apple Airpods Headphones,AA Batteries (4-pack)
3,AA Batteries (4-pack),Apple Airpods Headphones
4,Vareebadd Phone,Wired Headphones


In [16]:
#  Calculate how often each item A occurs with the items B
dataset_combo2 =dataset_combo.groupby(['A','B']).size()
dataset_combo2

A             B                     
20in Monitor  20in Monitor                6
              27in 4K Gaming Monitor      2
              27in FHD Monitor            5
              34in Ultrawide Monitor      2
              AA Batteries (4-pack)      16
                                       ... 
iPhone        ThinkPad Laptop             7
              USB-C Charging Cable       29
              Vareebadd Phone             7
              Wired Headphones          462
              iPhone                      4
Length: 334, dtype: int64

In [17]:
#create a sorted dataframe by the most frequent combinations.
dataset =dataset_combo2.reset_index()
dataset.columns = ['A','B',"Size"]
dataset.sort_values(by='Size',ascending =False, inplace =True)
dataset.head()

,A,B,Size
228,Lightning Charging Cable,iPhone,1015
327,iPhone,Lightning Charging Cable,1015
181,Google Phone,USB-C Charging Cable,999
273,USB-C Charging Cable,Google Phone,999
315,Wired Headphones,iPhone,462


In [18]:
#Clean the Product row and check the head of the data
Cleaned_Dataset = dataset[dataset["A"]!="Product"]
Cleaned_Dataset.head()

,A,B,Size
228,Lightning Charging Cable,iPhone,1015
327,iPhone,Lightning Charging Cable,1015
181,Google Phone,USB-C Charging Cable,999
273,USB-C Charging Cable,Google Phone,999
315,Wired Headphones,iPhone,462


In [19]:
#Create a combination of groups so that can be used as an alternative to pairing
df2=pd.read_csv('sales all data.csv')
df2= df2.dropna()
df2 =df2[df2['Order ID'].duplicated(keep=False)]
df2['Group'] = df2.groupby('Order ID')['Product'].transform(lambda x:','.join(x))
df2 =df2[['Order ID', 'Group']].drop_duplicates()
df2.head()


,Order ID,Group
3,176560.0,"Google Phone,Wired Headphones"
18,176574.0,"Google Phone,USB-C Charging Cable"
30,176585.0,"Bose SoundSport Headphones,Bose SoundSport Hea..."
32,176586.0,"AAA Batteries (4-pack),Google Phone"
119,176672.0,"Lightning Charging Cable,USB-C Charging Cable"


In [20]:
#Count each row combination by the the pairing of the two
count = Counter()
for row in df2['Group']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,2)))

print(count)

Counter({('iPhone', 'Lightning Charging Cable'): 1005, ('Google Phone', 'USB-C Charging Cable'): 987, ('iPhone', 'Wired Headphones'): 447, ('Google Phone', 'Wired Headphones'): 414, ('Vareebadd Phone', 'USB-C Charging Cable'): 361, ('iPhone', 'Apple Airpods Headphones'): 360, ('Google Phone', 'Bose SoundSport Headphones'): 220, ('USB-C Charging Cable', 'Wired Headphones'): 160, ('Vareebadd Phone', 'Wired Headphones'): 143, ('Lightning Charging Cable', 'Wired Headphones'): 92, ('Lightning Charging Cable', 'Apple Airpods Headphones'): 81, ('Vareebadd Phone', 'Bose SoundSport Headphones'): 80, ('USB-C Charging Cable', 'Bose SoundSport Headphones'): 77, ('Apple Airpods Headphones', 'Wired Headphones'): 69, ('Lightning Charging Cable', 'USB-C Charging Cable'): 58, ('Lightning Charging Cable', 'AA Batteries (4-pack)'): 55, ('Lightning Charging Cable', 'Lightning Charging Cable'): 54, ('Bose SoundSport Headphones', 'Wired Headphones'): 53, ('AA Batteries (4-pack)', 'Lightning Charging Cable')

In [23]:
#Top 10 most bought combination of items
for key, value in count.most_common(10):
    print(key,value)

('iPhone', 'Lightning Charging Cable') 1005
('Google Phone', 'USB-C Charging Cable') 987
('iPhone', 'Wired Headphones') 447
('Google Phone', 'Wired Headphones') 414
('Vareebadd Phone', 'USB-C Charging Cable') 361
('iPhone', 'Apple Airpods Headphones') 360
('Google Phone', 'Bose SoundSport Headphones') 220
('USB-C Charging Cable', 'Wired Headphones') 160
('Vareebadd Phone', 'Wired Headphones') 143
('Lightning Charging Cable', 'Wired Headphones') 92
